In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('')

In [ ]:
df.tail(10)

In [ ]:
df.shape

In [ ]:
#중복되는 열 찾아 제거
df.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
#date의 데이터 타입을 날짜타입으로 올바르게
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# 빈칸의 위치를 찾는 코드
df.isna().any()
df[df.isna().any(axis=1)].index

In [ ]:
# 빈칸을 과거 4일사이의 값들의 평균계산 반올림하여 메우는(lmputing) 코드
null = df[df.isna().any(axis=1).index].ignore_index
null = null[0]
df.loc[null, 'Leaflets'] = np.round(df.loc[null-4:null-1, 'Leaflets'].mean()) 

In [ ]:
#date컬럼을 이용해서 요일 만들기, dt.weekday 활용 , 진행시 integer로 encoding
df['day'] = df['Date'].dt.weekday

df['wdwe'] = np.where(df['day']>=5, 0, 1)

In [ ]:
# Leaflets 컬럼 x 축 , Lemon 컬럼을 y축으로 시작하는 산점도
plt.scatter(x='Lemon',y='Leaflets', data=df)
plt.show()

In [ ]:
# 레드에이드와 오렌지에이드의 판매량의 합계를 전단지 수로 나눈 값으로 effect_leaflets 생성
df['effect_leaflets'] = (df['Lemon']+df['Orange']) / ade['Leaflets']

In [ ]:
# 트레인 셋 , 테스트 셋 분힐
# y는 데이터 프레임의 Lemon과 Orange 컬럼이다
# x는 'Leaflets', 'price', 'effect_leaflets', 'wdwe'만 사용
# train: test  =  9:1
#Location의 Pack과 Beach가 골고루 분배되도록 stratify하게 만든다
# 변수명 x_train, x_test, y_train, y_test
#random state, seed 는 2021로 설정

target = 'Lemon'
x = df[['Leaflets', 'price', 'effect_leaflets', 'wdwe']]
y = df[target]

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=2021)

In [ ]:
# 트레이닝 셋(x_train, y_train['Lemon']) 이용 학습
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
#LinearRegression의 성능을 mean absolute error 성능 지표를 통해 확인
#test 데이터로 모델 예측 , 예측값 변수에는 y_pred를 사용
#성능 지표로는 mean absolute error를 사용
#성능 지표 출력

from sklearn.metrics import mean_absolute_error
y_pred = model.predict(x_test)
print(mean_absolute_error(y_test, y_pred))

In [ ]:
# 예측 데이터와 실제 데이터를 산점도를 통해서 정확성에 시각화
# scatter plot 이용
# 예측값 (y_pred)와 실제값(y_test)의 lemon항목에 대해 산점도를 그린다
# x, y축에 라벨을 붙여준다

plt.figure(figsize=(6,6))
plt.scatter(x=y_test,y=y_pred,alpha=0.6)
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Input  
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# BatchNormalization 사용, output layer는 Lemon 에이드 판매량 예측
# output layer의 예측값이 0미만의 값이 나올 수 없도록 activation을 지정한다
# EarlyStopping을 이용하여 100이상 개선되지 않으면 학습 중단, 가장 성능치가 좋을때 가중치 복구
# 힉습 과정의 로그를 history에 선언하여 남긴다. loss function은 mse 사용
# 에폭 200

In [ ]:
x_train.shape

In [ ]:
karse.backend.clear_session()

model = Sequential([           
	Input(shape=(5,)),
	Dense(16, activation='relu'),
	BatchNormalization(),
	Dense(8, activation='relu'),
	BatchNormalization(),
	Dense(1)
])

model.compile(loss='mse', optimizer='adam')
es = EarlyStopping(monitor='val_loss', restore_best_weights=True, min_delte=0, patience=100)

model.summary()

In [ ]:
hist = model.fit(x_train, y_train, epochs=200, callbacks=[es], validation_data=(x_valid, y_valid))

In [ ]:
plt.title('Loss')
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend() #범례
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.show()

In [ ]:
# test데이터를 활용하여 test 셋 데이터에 대해 mse 값을 출력
from sklearn.metrics import mean_absolute_error
y_pred = model.predict(x_test)
mean_absolute_error(y_test, y_pred)